In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign

In [2]:
# date0='0709_1157'
# date1='0709_1557'
# date2='0709_1934'
# dates=[date0,date1,date2]
# plate = 13
date0='0701_1957'
date1='0701_2357'
date2='0702_0357'
date3='0702_0757'
date4='0702_1157'
date5='0702_1557'
plate=13
dates=[date0,date1,date2,date3,date4,date5]

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [46]:
dic_skels[0]

<20685x45554 sparse matrix of type '<class 'numpy.float64'>'
	with 611408 stored elements in Compressed Sparse Column format>

In [47]:
dic_skels[1]

<20685x45554 sparse matrix of type '<class 'numpy.float64'>'
	with 634913 stored elements in Compressed Sparse Column format>

In [4]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [5]:
nx_graphs=[]
poss=[]
for tab in tabs_labeled:
    nx_graph,pos=generate_nx_graph(tab,labeled=True)
    nx_graphs.append(nx_graph)
    poss.append(pos)

In [6]:
nx_graph_clean=[]
for graph in nx_graphs:
    S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    len_connected=[len(nx_graph.nodes) for nx_graph in S]
    nx_graph_clean.append(S[np.argmax(len_connected)])

In [7]:
skeletons=[]
for nx_graph in nx_graph_clean:
    skeletons.append(generate_skeleton(nx_graph,dim=(20687, 45554)))

In [23]:
xbegin=18000
ybegin=8000
xend=22000
yend=13000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])

In [24]:
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [25]:
plot_t_tp1(node_smalls[1],node_smalls[2],poss[1],poss[2],skeletons_small[1].todense(),skeletons_small[2].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
def compress_skeleton(skeleton_doc,factor):
    shape=skeleton_doc.shape
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for pixel in skeleton_doc.keys():
        x=min(round(pixel[0]/factor),shape[0]//factor-1)
        y=min(round(pixel[1]/factor),shape[1]//factor-1)
        final_picture[x,y]+=1
    return(final_picture)

In [37]:
final_picture1 = compress_skeleton(skeletons[0],5)
final_picture2 = compress_skeleton(skeletons[1],5)

In [ ]:
lists_node = [[node for node in nx_graph_aligned.nodes if nx_graph_aligned.degree[node]==1] for nx_graph_aligned in nx_graphs_aligned]

In [38]:
plot_t_tp1([],[],poss[1],poss[2],final_picture1,final_picture2,compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
skeleton_docs_aligned = [sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date}_{plate}_full.npz')) for date in dates]

In [49]:
skeleton_docs_aligned[0]!=skeleton_docs_aligned[1]

<20687x45556 sparse matrix of type '<class 'numpy.bool_'>'
	with 0 stored elements in Compressed Sparse Row format>

In [39]:
final_picture1 = compress_skeleton(skeleton_docs_aligned[0],5)
final_picture2 = compress_skeleton(skeleton_docs_aligned[1],5)

In [40]:
plot_t_tp1([],[],poss[1],poss[2],final_picture1,final_picture2,compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [171]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
from util import get_path
from plotutil import show_im,overlap, show_im_rgb
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton
import networkx as nx
from copy import deepcopy
from sparse_util import dilate
from scipy.optimize import minimize
from node_id import orient
def whole_movement_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,length_id=50):
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    tip_origin={tip : tip  for tip in tips}
    pixels_from_tip={tip : [] for tip in tips}
    for number,tip in enumerate(tips):
#         print('tip',pos_tm1[tip],tip)
        if number%100==0:
            print(number/len(tips))
        mini=np.inf
        for edge in nx_graph_t.edges:
            pixel_list=nx_graph_t.get_edge_data(*edge)['pixel_list']
            if np.linalg.norm(np.array(pixel_list[0])-np.array(pos_tm1[tip]))<=5000:
                distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos_tm1[tip]),axis=1))
                if distance<mini:
                    mini=distance
                    right_edge = edge
        origin = np.array(orient(nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['pixel_list'],pos_tm1[tip]))
        origin_vector = origin[0]-origin[-1]
        branch=np.array(orient(nx_graph_t.get_edge_data(*right_edge)['pixel_list'],pos_t[right_edge[0]]))
        index_nearest_pixel=np.argmin(np.linalg.norm(branch-np.array(pos_tm1[tip]),axis=1))
        candidate_vector = branch[-1]-branch[0]
        dot_product = np.dot(origin_vector,candidate_vector)
#         if tip==5260:
#             print(list(branch[index_nearest_pixel:]))
#             print(list(branch[:index_nearest_pixel]))
        if dot_product>=0:
            root=right_edge[0]
            next_node=right_edge[1]
            pixels_from_tip[tip]+=list(branch[index_nearest_pixel:])
        else:
            root=right_edge[1]
            next_node=right_edge[0]
            pixels_from_tip[tip]+=list(reversed(list(branch[:index_nearest_pixel])))
        #Could improve the candidate vector by chosing pixel around the forme tip but this identification should be rather unambiguous
        last_node=root
        current_node=next_node
        last_branch=np.array(orient(nx_graph_t.get_edge_data(root,next_node)['pixel_list'],pos_t[current_node]))
        def label_node_recursive(last_node,current_node,corresp_label):
            if not current_node in corresp_label.keys() and not current_node in nx_graph_tm1.nodes:
                corresp_label[current_node]=tip
                pixel_list=nx_graph_t.get_edge_data(last_node,current_node)['pixel_list']
                pixels_from_tip[tip]+=pixel_list
                if nx_graph_t.degree(current_node)>=3:
                    for neighbour_t in nx_graph_t.neighbors(current_node): 
                        if neighbour_t!=last_node:
                            label_node_recursive(current_node,neighbour_t,corresp_label)
        label_node_recursive(last_node,current_node,tip_origin)
    return(tip_origin,pixels_from_tip)

In [172]:
tip_origin_t0m2_t0m1,pixels_from_tip= whole_movement_identification(nx_graphs_aligned[3],nx_graphs_aligned[4],poss_aligned[3],poss_aligned[4])

0.0
0.0544069640914037
0.1088139281828074
0.1632208922742111
0.2176278563656148
0.2720348204570185
0.3264417845484222
0.3808487486398259
0.4352557127312296
0.4896626768226333
0.544069640914037
0.5984766050054406
0.6528835690968444
0.7072905331882481
0.7616974972796517
0.8161044613710555
0.8705114254624592
0.9249183895538629
0.9793253536452666


In [ ]:
def from_nx_to_tab(nx_graph,pos,tip_origin_corresp,pixel_from_tip):
    column_names = ["origin_label","end_label","origin_pos", "end_pos", "pixel_list"]
    tab = pd.DataFrame(columns=column_names)
    for edge in nx_graph.edges:
        origin_label=edge[0]
        end_label=edge[1]
        origin_pos=pos[origin_label]
        end_pos = pos[end_label]
        pixel_list=orient(nx_graph.get_edge_data(*edge)['pixel_list'],origin_pos)
        new_line=pd.DataFrame({"origin_label":[origin_label],"end_label" : [end_label], "origin_pos":[origin_pos], "end_pos" : [end_pos], "pixel_list" : [pixel_list]})
        tab=tab.append(new_line,ignore_index=True)
    return(tab)

In [223]:
node_problem=4607
pos_problem=poss_aligned[0][node_problem]
xbegin=pos_problem[0]-1000
ybegin=pos_problem[1]-1000
xend=pos_problem[0]+1000
yend=pos_problem[1]+1000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graphs_aligned):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss_aligned[i][node][0]<xend and ybegin<poss_aligned[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [224]:
plot_t_tp1([node_problem],[node_problem],poss_aligned[0],poss_aligned[1],skeletons_small[0].todense(),skeletons_small[1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [202]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(growths,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.213e+03, 2.960e+02, 1.360e+02, 6.400e+01, 3.300e+01, 1.700e+01,
        1.100e+01, 4.000e+00, 7.000e+00, 1.000e+00, 1.000e+00, 2.000e+00,
        1.000e+00, 1.000e+00, 5.000e+00, 1.000e+00, 0.000e+00, 2.000e+00,
        2.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 2.000e+00, 0.000e+00,
        2.000e+00, 1.000e+00, 2.000e+00, 2.000e+00, 4.000e+00, 1.000e+00]),
 array([ 0. ,  3.2,  6.4,  9.6, 12.8, 16. , 19.2, 22.4, 25.6, 28.8, 32. ,
        35.2, 38.4, 41.6, 44.8, 48. , 51.2, 54.4, 57.6, 60.8, 64. , 67.2,
        70.4, 73.6, 76.8, 80. , 83.2, 86.4, 89.6, 92.8, 96. ]),
 <BarContainer object of 30 artists>)

In [238]:
def make_growth_picture(pixels_from_tip,pos,shape=(20700,45600),factor=10):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        beginx=round(pos[tip][0]/factor-factor//2)
        endx=round(pos[tip][0]/factor+factor//2)
        beginy=round(pos[tip][1]/factor-factor//2)
        endy=round(pos[tip][1]/factor+factor//2)
        if growth<=200:
            final_picture[beginx:endx,beginy:endy]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    return(final_picture)

In [361]:
def make_growth_picture_per_tip(pixels_from_tip,pos,shape=(20700,45600),factor=10,max_growth=200,min_growth=10,per_tip=True):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    number_tips = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        x=min(round(pos[tip][0]/factor),shape[0]//factor-1)
        y=min(round(pos[tip][1]/factor),shape[1]//factor-1)
        if growth<=max_growth:
#             print(number_tips)
            if growth>=min_growth:
                number_tips[x,y]+=1
                final_picture[x,y]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    if per_tip:
        return(final_picture/(number_tips+(number_tips==0).astype(np.int)),number_tips)
    else:
        return(final_picture,number_tips)

In [3]:
skeleton_docs_aligned = [sparse.dok_matrix(sparse.load_npz(f'Data/skeleton_{date}_{plate}_full.npz')) for date in dates]

In [ ]:
nx_graph_poss=[generate_nx_graph_from_skeleton(skeleton) for skeleton in skeleton_docs_aligned]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]

In [43]:
final_picture1 = compress_skeleton(skeleton_docs_aligned[0],5)
final_picture2 = compress_skeleton(skeleton_docs_aligned[3],5)

In [26]:
lists_node = [[node for node in nx_graph_aligned.nodes if nx_graph_aligned.degree[node]==1] for nx_graph_aligned in nx_graphs_aligned]

In [44]:
plot_t_tp1([],[],poss_aligned[0],poss_aligned[3],final_picture1,final_picture2,compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
final_picture3 = compress_skeleton(skeleton_docs[0],5)
final_picture4 = compress_skeleton(skeleton_docs[3],5)
plot_t_tp1([],[],poss_aligned[0],poss_aligned[3],final_picture3,final_picture4,compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]
skeleton_docs=[sparse.dok_matrix(dic_skel) for dic_skel in dic_skels]

In [9]:
plot_t_tp1([],[],poss_aligned[1],poss_aligned[2],final_picture1,final_picture2,compress=10)

NameError: name 'poss_aligned' is not defined

In [412]:
final_picture1 = compress_skeleton(skeleton_docs_aligned[0],10)
final_picture2 = compress_skeleton(skeleton_docs_aligned[2],10)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(final_picture1>0,cmap='gray',interpolation='none')
ax.imshow(final_picture2>0,cmap='jet',alpha=0.5,interpolation='none')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2 
import numpy as np


In [398]:
final_picture,number_tips = make_growth_picture_per_tip(pixels_from_tip,poss_aligned[0],factor=300,max_growth=300,per_tip=True,min_growth=0)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(final_picture)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [388]:
26*500,14*500

(13000, 7000)

In [389]:
pos_problem=(13000, 7000)
xbegin=pos_problem[0]-2000
ybegin=pos_problem[1]-2000
xend=pos_problem[0]+2000
yend=pos_problem[1]+2000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graphs_aligned):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss_aligned[i][node][0]<xend and ybegin<poss_aligned[i][node][1]<yend and nx_graph.degree(node)==1)])

In [390]:
plot_t_tp1(node_smalls[0],node_smalls[1],poss_aligned[0],poss_aligned[1],skeletons_small[0].todense(),skeletons_small[1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:
poss_aligned[1][5136]

array([19343, 33314])

In [315]:
len(pixels_from_tip[27])

197